In [8]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import swifter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm
import torch.nn.functional as F
import torch
from torch.nn.functional import softmax

In [9]:
csv_folder = r"C:\Users\jiali\Documents\Studium\DM-Dataset"
csv_path = os.path.join(csv_folder, "posts_with_comments_and_users.csv")
df = pd.read_csv(csv_path)

C:\Users\jiali\AppData\Local\Temp\ipykernel_36412\1934984706.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [10]:
df

,PostId,posts.PostTypeId,posts.AcceptedAnswerId,posts.CreationDate,posts.Score,posts.ViewCount,posts.Body,posts.OwnerUserId,posts.LastEditorUserId,posts.LastEditDate,...,users.Location,users.AboutMe,users.Views,users.UpVotes,users.DownVotes,users.AccountId,users.Country,users.SentimentLabel,users.SentimentScore,users.SentimentValue
0,1,1,5.0,2010-11-02 18:59:01.140,87,20982.0,I often hear about subatomic particles having ...,7.0,520.0,2012-08-19 23:00:18.417,...,NaN,NaN,708.0,133.0,11.0,510285.0,NaN,NaN,NaN,NaN
1,1,1,5.0,2010-11-02 18:59:01.140,87,20982.0,I often hear about subatomic particles having ...,7.0,520.0,2012-08-19 23:00:18.417,...,switzerland,NaN,15.0,25.0,1.0,209557.0,switzerland,NaN,NaN,NaN
2,1,1,5.0,2010-11-02 18:59:01.140,87,20982.0,I often hear about subatomic particles having ...,7.0,520.0,2012-08-19 23:00:18.417,...,europe,I'm a theoretical physicist.,120311.0,9277.0,9460.0,387099.0,NaN,neutral,0.765797,0.0
3,2,1,5704.0,2010-11-02 19:09:48.680,31,3946.0,How would you explain string theory to non-phy...,23.0,58382.0,2020-06-14 12:52:57.027,...,"london, united kingdom","entrepreneur; graduate in mathematics, logic, ...",2182.0,151.0,33.0,19078.0,united kingdom,neutral,0.783226,0.0
4,2,1,5704.0,2010-11-02 19:09:48.680,31,3946.0,How would you explain string theory to non-phy...,23.0,58382.0,2020-06-14 12:52:57.027,...,NaN,NaN,708.0,133.0,11.0,510285.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339924,808520,1,808552.0,2024-03-31 23:05:36.567,1,66.0,For simplicity I will be considering $\phi^4$ ...,288281.0,2451.0,2024-04-01 01:26:51.400,...,"amsterdam, nl",My research is broadly in the field of quantum...,7675.0,539.0,185.0,1425173.0,netherlands,neutral,0.800259,0.0
1339925,808520,1,808552.0,2024-03-31 23:05:36.567,1,66.0,For simplicity I will be considering $\phi^4$ ...,288281.0,2451.0,2024-04-01 01:26:51.400,...,NaN,NaN,325.0,171.0,1.0,19486123.0,NaN,NaN,NaN,NaN
1339926,808520,1,808552.0,2024-03-31 23:05:36.567,1,66.0,For simplicity I will be considering $\phi^4$ ...,288281.0,2451.0,2024-04-01 01:26:51.400,...,"amsterdam, nl",My research is broadly in the field of quantum...,7675.0,539.0,185.0,1425173.0,netherlands,neutral,0.800259,0.0
1339927,808520,1,808552.0,2024-03-31 23:05:36.567,1,66.0,For simplicity I will be considering $\phi^4$ ...,288281.0,2451.0,2024-04-01 01:26:51.400,...,NaN,NaN,325.0,171.0,1.0,19486123.0,NaN,NaN,NaN,NaN


In [11]:
df.columns

Index(['PostId', 'posts.PostTypeId', 'posts.AcceptedAnswerId',
       'posts.CreationDate', 'posts.Score', 'posts.ViewCount', 'posts.Body',
       'posts.OwnerUserId', 'posts.LastEditorUserId', 'posts.LastEditDate',
       'posts.LastActivityDate', 'posts.Title', 'posts.Tags',
       'posts.AnswerCount', 'posts.CommentCount', 'posts.ContentLicense',
       'posts.ParentId', 'posts.OwnerDisplayName', 'posts.ClosedDate',
       'posts.LastEditorDisplayName', 'posts.CommunityOwnedDate',
       'posts.FavoriteCount', 'posts.CleanBodyNoMath', 'posts.SentimentScore',
       'posts.SentimentLabel', 'CommentId', 'comments.Score', 'comments.Text',
       'comments.CreationDate', 'UserId', 'comments.UserDisplayName',
       'users.Reputation', 'users.CreationDate', 'users.DisplayName',
       'users.LastAccessDate', 'users.WebsiteUrl', 'users.Location',
       'users.AboutMe', 'users.Views', 'users.UpVotes', 'users.DownVotes',
       'users.AccountId', 'users.Country', 'users.SentimentLabel',
  

In [12]:
csv_path = os.path.join(csv_folder, "Posts_with_vote_counts.csv")
df1 = pd.read_csv(csv_path)

In [13]:
df1

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Favorite_count,Close_count,Reopen_count,BountyStart_count,BountyClose_count,Deletion_count,Undeletion_count,Spam_count,ModeratorReview_count,ApprovedEditSuggestion_count
0,1,1,5,2010-11-02 18:59:01.140,87,20982,I often hear about subatomic particles having ...,7,520,2012-08-19 23:00:18.417,...,0,0,0,0,0,0,0,0,0,0
1,2,1,5704,2010-11-02 19:09:48.680,31,3946,How would you explain string theory to non-phy...,23,58382,2020-06-14 12:52:57.027,...,0,0,0,0,0,0,0,0,0,1
2,3,1,180,2010-11-02 19:11:05.857,46,3308,This is a question that has been posted at man...,40,2451,2013-01-28 22:30:52.200,...,0,0,0,0,0,0,0,0,0,0
3,4,2,0,2010-11-02 19:12:26.543,28,0,Imagine going to the rest frame of a massive p...,20,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2,0,2010-11-02 19:15:20.813,52,0,Spin is a technical term specifically referrin...,13,75633,2017-12-13 08:24:54.477,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576953,808517,2,0,2024-03-31 22:43:15.067,1,0,Is the first trace in the question title a tra...,352117,352117,2024-04-01 15:39:37.470,...,0,0,0,0,0,0,0,0,0,0
576954,808518,1,0,2024-03-31 22:49:37.613,1,32,Suppose we have a theory that is super-renorma...,288281,2451,2024-04-01 01:01:53.040,...,0,0,0,0,0,0,0,0,0,0
576955,808519,1,0,2024-03-31 22:59:29.267,0,32,What is the speed of light in a vacuum when th...,198517,2451,2024-04-01 00:57:51.470,...,0,0,0,0,0,0,0,0,0,0
576956,808520,1,808552,2024-03-31 23:05:36.567,1,66,For simplicity I will be considering $\phi^4$ ...,288281,2451,2024-04-01 01:26:51.400,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df1.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'ContentLicense', 'ParentId', 'OwnerDisplayName', 'ClosedDate',
       'LastEditorDisplayName', 'CommunityOwnedDate', 'FavoriteCount',
       'CleanBodyNoMath', 'SentimentScore', 'SentimentLabel',
       'AcceptedByOrginator_count', 'Upvote_count', 'Downvote_count',
       'Offensive_count', 'Favorite_count', 'Close_count', 'Reopen_count',
       'BountyStart_count', 'BountyClose_count', 'Deletion_count',
       'Undeletion_count', 'Spam_count', 'ModeratorReview_count',
       'ApprovedEditSuggestion_count'],
      dtype='object')

In [15]:
df1.rename(columns={'Id': 'PostId'}, inplace=True)


In [16]:
columns_to_keep = [
    'PostId', 'AcceptedByOrginator_count', 'Upvote_count', 'Downvote_count',
    'Offensive_count', 'Favorite_count', 'Close_count', 'Reopen_count',
    'BountyStart_count', 'BountyClose_count', 'Deletion_count',
    'Undeletion_count', 'Spam_count', 'ModeratorReview_count',
    'ApprovedEditSuggestion_count'
]

df1 = df1[columns_to_keep]

In [18]:
df1.columns

Index(['PostId', 'AcceptedByOrginator_count', 'Upvote_count', 'Downvote_count',
       'Offensive_count', 'Favorite_count', 'Close_count', 'Reopen_count',
       'BountyStart_count', 'BountyClose_count', 'Deletion_count',
       'Undeletion_count', 'Spam_count', 'ModeratorReview_count',
       'ApprovedEditSuggestion_count'],
      dtype='object')

In [19]:
merged_df = pd.merge(df, df1, on='PostId', how='left')


In [20]:
merged_df.columns

Index(['PostId', 'posts.PostTypeId', 'posts.AcceptedAnswerId',
       'posts.CreationDate', 'posts.Score', 'posts.ViewCount', 'posts.Body',
       'posts.OwnerUserId', 'posts.LastEditorUserId', 'posts.LastEditDate',
       'posts.LastActivityDate', 'posts.Title', 'posts.Tags',
       'posts.AnswerCount', 'posts.CommentCount', 'posts.ContentLicense',
       'posts.ParentId', 'posts.OwnerDisplayName', 'posts.ClosedDate',
       'posts.LastEditorDisplayName', 'posts.CommunityOwnedDate',
       'posts.FavoriteCount', 'posts.CleanBodyNoMath', 'posts.SentimentScore',
       'posts.SentimentLabel', 'CommentId', 'comments.Score', 'comments.Text',
       'comments.CreationDate', 'UserId', 'comments.UserDisplayName',
       'users.Reputation', 'users.CreationDate', 'users.DisplayName',
       'users.LastAccessDate', 'users.WebsiteUrl', 'users.Location',
       'users.AboutMe', 'users.Views', 'users.UpVotes', 'users.DownVotes',
       'users.AccountId', 'users.Country', 'users.SentimentLabel',
  

In [ ]:
merged_df

In [21]:
merged_df.to_csv('merged_output.csv', index=False)
